In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext sql 
%pylab inline
%matplotlib inline

import pandas as pd
import os, random

import pickle,time

tmstmp = time.strftime("%m-%d-%Y",time.localtime())


Populating the interactive namespace from numpy and matplotlib


# Initialisation

## Directory paths
* TOP = path to the toplevel directory 
* TOP+/lib = path to the python source files that are required
* FIG_DIR = path to where figures are stored

In [2]:
# Set up the local source files
TOP = os.getcwd().replace('notebooks','')
LIB = TOP+'lib'
if not LIB in sys.path: sys.path.append(LIB)

DAT_DIR = TOP + '/data/'
RES_DIR = TOP + '/results/'
FIG_DIR = TOP + '/figs/'


## Load packages
Open organtox_v1 MongoDB - if this fails then check your mongodb installation

In [3]:
from organtox import *

DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='organtox_v1')
GNR = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='genra_v3')
DB.collection_names()

/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[u'chm_fp',
 u'ml_lr_v1',
 u'ml_study_lr_v1',
 u'bio_fp',
 u'ml_summary_v2',
 u'ml_study_summary_v1',
 u'ml_summary_v1',
 u'ml_run_v1',
 u'tox_fp',
 u'ml_study_run_v1',
 u'tox2_fp']

# Toxicity data

Extend the toxicity fp to include only study level hits

In [ ]:
GNR.tox_fp.find_one()

In [ ]:
DB.tox_fp.find_one()

## Build study level descriptors
1. Create collection tox2_fp
2. Define tox_fpp2: study is positive if tox_fp.tox_fpp1 has STUDY
3. Define tox_fpn2: study is negative if tox_fp.tox_fpn1 has STUDY and not tox_fpp2


In [28]:
from util.jupyter import *

DB.tox2_fp.drop()

for FP in log_progress(DB.tox_fp.find({},dict(_id=0)),size=DB.tox_fp.count()):
    FPN1 = FP.pop('tox_fpn1')
    FPP1 = FP.pop('tox_fpp1')
    
    Pos = set([i.split(':')[0] for i in FPP1['ds']])
    Neg = set([i.split(':')[0] for i in FPN1['ds']])
    Neg2= list(Neg.difference(Pos))
    
    FPP2= dict(ds=list(Pos),n=len(Pos))
    FPN2= dict(ds=list(Neg2),n=len(Neg2))
    
    FP['tox_fpp2']=FPP2
    FP['tox_fpn2']=FPN2
    
    # Dose
    X = GNR.tox_fp.find_one(dict(dsstox_cid=FP['dsstox_cid']),dict(_id=0,toxq1=1))
    if X:
        Toxq = pd.DataFrame(X['toxq1'].items(),columns=['effect','dose'])
        Toxq['study']=Toxq.effect.str[:3]
        Toxq['dose']=Toxq.dose.astype(np.float)
        
        Dose=Toxq.query("dose>0").groupby('study').aggregate(dict(dose=dict(mn=np.min,md=np.median,mx=np.max)))
        C = [i+'_'+j for i,j in list(Dose.columns)]
        Dose.columns=C
        FP['toxq2']=Dose.reset_index().to_dict('records')
        
    DB.tox2_fp.insert_one(FP)

In [29]:
DB.tox2_fp.find_one({'tox_fpp2.n':{'$gte':3}})

{u'_id': ObjectId('59a5965b072e601d90cd5f4c'),
 u'dsstox_cid': u'DTXCID10194',
 u'tox_fpn2': {u'ds': [], u'n': 0},
 u'tox_fpp2': {u'ds': [u'MGR', u'CHR', u'SUB', u'DEV'], u'n': 4},
 u'toxq2': [{u'dose_md': 167.5,
   u'dose_mn': 167.5,
   u'dose_mx': 1100.0,
   u'study': u'CHR'},
  {u'dose_md': 205.5, u'dose_mn': 76.0, u'dose_mx': 1003.0, u'study': u'DEV'},
  {u'dose_md': 4500.0,
   u'dose_mn': 336.0,
   u'dose_mx': 4500.0,
   u'study': u'MGR'},
  {u'dose_md': 1025.4,
   u'dose_mn': 25.0,
   u'dose_mx': 4466.7,
   u'study': u'SUB'}]}

## See if this builds correct datasets

In [48]:
from organtox import *

DS1 = getToxDataSet('DEV',MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')

In [49]:
[[k,v.shape] for k,v in DS1.iteritems()]

[['bio', (429, 780)],
 ['tox', (581, 1)],
 ['bc', (424, 2782)],
 ['bct', (424, 1151)],
 ['chm', (476, 2002)],
 ['ct', (424, 371)]]

# ML Using XGBoost

In [10]:
from organtox import *

tox = 'CHR'
DS1 = getToxDataSet(tox,MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')


In [19]:
dt = 'bio'
I = DS1['tox'].index.intersection(DS1[dt].index)
Xi,Yi = DS1[dt].ix[I], DS1['tox'].ix[I]
Fi = Xi.sum(axis=0)/Xi.shape[0]
Fi[Fi>0.1.shape

SyntaxError: invalid syntax (<ipython-input-19-b2876ac1e70b>, line 5)

In [36]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit,LeaveOneOut
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,make_scorer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import copy,warnings

from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,make_scorer

def ppv_calc(y,yp):
    TP = len(np.where(np.logical_and(y==yp,y==1))[0])
    TN = len(np.where(np.logical_and(y==yp,y==0))[0])
    FP = len(np.where(np.logical_and(yp==1,y==0))[0])
    FN = len(np.where(np.logical_and(yp==0,y==1))[0])
    v = None
    try:
        v = 1.0*TP/(TP+FP)
    except:
        v = 0
    return v

ppv_scorer = make_scorer(ppv_calc,greater_is_better=True)
f1_scorer = make_scorer(f1_score,greater_is_better=True)
sn_scorer = make_scorer(recall_score,greater_is_better=True)
sp_scorer = make_scorer(precision_score,greater_is_better=True)

PERF=[]

for tox in DB.tox2_fp.distinct('tox_fpp2.ds'):
    DS1 = getToxDataSet(tox,MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')
    N_p,N_n = (DS1['tox'][tox]==1).sum(),(DS1['tox'][tox]==0).sum()
    if N_p==0 or N_n ==0: continue
        
    for dt in DS1.keys():
        if dt == 'tox': continue
        I = DS1['tox'].index.intersection(DS1[dt].index)
        Xi,Yi = DS1[dt].ix[I], DS1['tox'].ix[I]
        Fi = Xi.sum(axis=0)/Xi.shape[0]
        Xi = Xi[Fi[Fi>0.05].index]
        P = dict(cls=tox,dti=dt,cv='loocv')
        print "\t %(cls)s %(dti)s" % P, Xi.shape,N_p,N_n
        PL0 = Pipeline([('fsel',SelectKBest(f_classif, k=12)),
                        ('svc',SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0, 
                           degree=3,gamma=10.0, kernel='rbf', max_iter=100, 
                           probability=False, shrinking=True,
                           tol=0.001, verbose=False))])

        F1_sc = cross_val_score(PL0,Xi,Yi, 
                                cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
                                n_jobs=50,
                                scoring=f1_scorer)
        Sn_sc = cross_val_score(PL0,Xi,Yi, 
                                cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
                                n_jobs=50,
                                scoring=sn_scorer)
        Sp_sc = cross_val_score(PL0,Xi,Yi, 
                                cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
                                n_jobs=50,
                                scoring=sp_scorer)
        #PPV_sc= cross_val_score(PL0,Xi,Yi, 
        #                        cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
        #                        n_jobs=50,
        #                        scoring=ppv_scorer)
        P.update(dict(f1_mn=np.mean(F1_sc),f1_sd=np.std(F1_sc),
                      sn_mn=np.mean(Sn_sc),sn_sd=np.std(Sn_sc),
         #             ppv_mn=np.mean(PPV_sc),ppv_sd=np.std(PPV_sc),
                      sp_mn=np.mean(Sp_sc),sp_sd=np.std(Sp_sc)
                     ))
        print "\t    F1 %(f1_mn)3.2f(%(f1_sd)3.2f)" % P
        #print "\t    PPV %(ppv_mn)3.2f(%(ppv_sd)3.2f)\n" % P
        PERF.append(P)
    

	 SUB bio (398, 373) 496 31
	    F1 0.90(0.03)
	 SUB bc (392, 476) 496 31
	    F1 0.63(0.24)
	 SUB bct (392, 433) 496 31
	    F1 0.83(0.13)
	 SUB chm (437, 114) 496 31
	    F1 0.27(0.23)
	 SUB ct (392, 68) 496 31
	    F1 0.33(0.28)
	 CHR bio (428, 369) 520 19
	    F1 0.90(0.02)
	 CHR bc (421, 471) 520 19
	    F1 0.90(0.03)
	 CHR bct (421, 435) 520 19
	    F1 0.93(0.03)
	 CHR chm (474, 109) 520 19
	    F1 0.79(0.23)
	 CHR ct (421, 67) 520 19
	    F1 0.53(0.41)
	 DEV bio (429, 376) 526 55
	    F1 0.12(0.15)
	 DEV bc (424, 481) 526 55
	    F1 0.22(0.31)
	 DEV bct (424, 439) 526 55
	    F1 0.27(0.29)
	 DEV chm (476, 115) 526 55
	    F1 0.05(0.10)
	 DEV ct (424, 64) 526 55
	    F1 0.06(0.10)
	 DNT bio (79, 423) 92 19
	    F1 0.84(0.15)
	 DNT bc (76, 588) 92 19
	    F1 0.78(0.17)
	 DNT bct (76, 478) 92 19
	    F1 0.82(0.19)
	 DNT chm (100, 114) 92 19
	    F1 0.71(0.12)
	 DNT ct (76, 61) 92 19
	    F1 0.84(0.08)
	 MGR bio (346, 383) 338 101
	    F1 0.50(0.21)
	 MGR bc (339, 508) 338 101
	    

In [39]:
Perf1 = pd.DataFrame(PERF)

Perf1['cv']='KF5'
Perf1['lr']='SVCR'
Perf1.head()

,cls,cv,dti,f1_mn,f1_sd,sn_mn,sn_sd,sp_mn,sp_sd,lr
0,SUB,KF5,bio,0.904670,0.031403,0.878739,0.056611,0.934180,0.010015,SVCR
1,SUB,KF5,bc,0.633282,0.237516,0.534247,0.276699,0.931217,0.040767,SVCR
2,SUB,KF5,bct,0.827951,0.134460,0.767123,0.189814,0.939953,0.016419,SVCR
3,SUB,KF5,chm,0.269961,0.227987,0.177055,0.151972,0.580000,0.474974,SVCR
4,SUB,KF5,ct,0.325608,0.284961,0.235616,0.214750,0.567136,0.463323,SVCR


In [41]:
Perf1['res2']=Perf1.apply(lambda x: "%s:%s/%s/%s" % (x.cls,x.lr,x.dti,x.cv),axis=1)
Perf1['res2_f1']=Perf1.apply(lambda x: "%3.2f±%3.2f" % (x.f1_mn,x.f1_sd),axis=1)
Perf1['res2_sn']=Perf1.apply(lambda x: "%3.2f±%3.2f" % (x.sn_mn,x.sn_sd),axis=1)
Perf1['res2_sp']=Perf1.apply(lambda x: "%3.2f±%3.2f" % (x.sp_mn,x.sp_sd),axis=1)
#Perf1['res2_ppv']=Perf1.apply(lambda x: "%3.2f±%3.2f" % (x.ppv_mn,x.ppv_sd),axis=1)
Perf1.head()

,cls,cv,dti,f1_mn,f1_sd,sn_mn,sn_sd,sp_mn,sp_sd,lr,res2,res2_f1,res2_sn,res2_sp
0,SUB,KF5,bio,0.904670,0.031403,0.878739,0.056611,0.934180,0.010015,SVCR,SUB:SVCR/bio,0.90±0.03,0.88±0.06,0.93±0.01
1,SUB,KF5,bc,0.633282,0.237516,0.534247,0.276699,0.931217,0.040767,SVCR,SUB:SVCR/bc,0.63±0.24,0.53±0.28,0.93±0.04
2,SUB,KF5,bct,0.827951,0.134460,0.767123,0.189814,0.939953,0.016419,SVCR,SUB:SVCR/bct,0.83±0.13,0.77±0.19,0.94±0.02
3,SUB,KF5,chm,0.269961,0.227987,0.177055,0.151972,0.580000,0.474974,SVCR,SUB:SVCR/chm,0.27±0.23,0.18±0.15,0.58±0.47
4,SUB,KF5,ct,0.325608,0.284961,0.235616,0.214750,0.567136,0.463323,SVCR,SUB:SVCR/ct,0.33±0.28,0.24±0.21,0.57±0.46


In [50]:
Table1 = Perf1[['res2','cls','dti','lr','res2_f1','res2_sn','res2_sp']]
Table1.columns=['Classifier id','Toxicity','dt','Algorithm','F1 Score','Sensitivity','Specificity']
#Table1.to_excel(DAT_DIR+'Table-1.xlsx',encoding='utf8')
Table1.sort_values(['Toxicity','dt','F1 Score']).set_index(['Toxicity','dt'])

Classifier id Algorithm   F1 Score Sensitivity Specificity
Toxicity dt                                                            
CHR      bc    CHR:SVCR/bc      SVCR  0.90±0.03   0.85±0.06   0.97±0.01
         bct  CHR:SVCR/bct      SVCR  0.93±0.03   0.88±0.05   0.98±0.01
         bio  CHR:SVCR/bio      SVCR  0.90±0.02   0.84±0.04   0.97±0.01
         chm  CHR:SVCR/chm      SVCR  0.79±0.23   0.72±0.27   0.97±0.01
         ct    CHR:SVCR/ct      SVCR  0.53±0.41   0.47±0.37   0.78±0.39
DEV      bc    DEV:SVCR/bc      SVCR  0.22±0.31   0.18±0.28   0.36±0.44
         bct  DEV:SVCR/bct      SVCR  0.27±0.29   0.21±0.27   0.54±0.44
         bio  DEV:SVCR/bio      SVCR  0.12±0.15   0.07±0.09   0.35±0.43
         chm  DEV:SVCR/chm      SVCR  0.05±0.10   0.03±0.06   0.17±0.34
         ct    DEV:SVCR/ct      SVCR  0.06±0.10   0.04±0.06   0.38±0.47
DNT      bc    DNT:SVCR/bc      SVCR  0.78±0.17   0.77±0.28   0.88±0.07
         bct  DNT:SVCR/bct      SVCR  0.82±0.19   0.85±0.27   0.84±0.03
         bio  DNT:SVCR/bio      SVCR  0.84±0.15   0.85±0.22   0.86±0.08
         chm  DNT:SVCR/chm      SVCR  0.71±0.12   0.66±0.16   0.77±0.06
         ct    DNT:SVCR/ct      SVCR  0.84±0.08   0.85±0.14   0.84±0.02
MGR      bc    MGR:SVCR/bc      SVCR  0.28±0.18   0.19±0.13   0.63±0.31
         bct  MGR:SVCR/bct      SVCR  0.41±0.14   0.30±0.11   0.74±0.06
         bio  MGR:SVCR/bio      SVCR  0.50±0.21   0.44±0.28   0.72±0.05
         chm  MGR:SVCR/chm      SVCR  0.46±0.36   0.48±0.44   0.65±0.34
         ct    MGR:SVCR/ct      SVCR  0.15±0.16   0.10±0.11   0.36±0.30
SUB      bc    SUB:SVCR/bc      SVCR  0.63±0.24   0.53±0.28   0.93±0.04
         bct  SUB:SVCR/bct      SVCR  0.83±0.13   0.77±0.19   0.94±0.02
         bio  SUB:SVCR/bio      SVCR  0.90±0.03   0.88±0.06   0.93±0.01
         chm  SUB:SVCR/chm      SVCR  0.27±0.23   0.18±0.15   0.58±0.47
         ct    SUB:SVCR/ct      SVCR  0.33±0.28   0.24±0.21   0.57±0.46

## Randomise the data

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit,LeaveOneOut
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,make_scorer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import copy,warnings

from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,make_scorer

def ppv_calc(y,yp):
    TP = len(np.where(np.logical_and(y==yp,y==1))[0])
    TN = len(np.where(np.logical_and(y==yp,y==0))[0])
    FP = len(np.where(np.logical_and(yp==1,y==0))[0])
    FN = len(np.where(np.logical_and(yp==0,y==1))[0])
    v = None
    try:
        v = 1.0*TP/(TP+FP)
    except:
        v = 0
    return v

ppv_scorer = make_scorer(ppv_calc,greater_is_better=True)
f1_scorer = make_scorer(f1_score,greater_is_better=True)
sn_scorer = make_scorer(recall_score,greater_is_better=True)
sp_scorer = make_scorer(precision_score,greater_is_better=True)

PERF_r=[]

for tox in DB.tox2_fp.distinct('tox_fpp2.ds'):
    DS1 = getToxDataSet(tox,MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')
    N_p,N_n = (DS1['tox'][tox]==1).sum(),(DS1['tox'][tox]==0).sum()
    if N_p==0 or N_n ==0: continue
        
    for dt in DS1.keys():
        if dt == 'tox': continue
        I = DS1['tox'].index.intersection(DS1[dt].index)
        Xi,Yi = DS1[dt].ix[I], DS1['tox'].ix[I]
        Fi = Xi.sum(axis=0)/Xi.shape[0]
        Xi = Xi[Fi[Fi>0.05].index]
        
        neg_f = 1.0*N_n/(N_n+N_p)
        Yr = Yi.copy()
        Ys = Yi.sample(frac=neg_f)
        Yr.ix[Ys.index]=0
        Yr.ix[Yr.index.difference(Ys.index)]=1
        
        P = dict(cls=tox,dti=dt,cv='loocv')
        print "\t %(cls)s %(dti)s" % P, Xi.shape,N_p,N_n
        PL0 = Pipeline([('fsel',SelectKBest(f_classif, k=12)),
                        ('svc',SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0, 
                           degree=3,gamma=10.0, kernel='rbf', max_iter=100, 
                           probability=False, shrinking=True,
                           tol=0.001, verbose=False))])

        F1_sc = cross_val_score(PL0,Xi,Yr, 
                                cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
                                n_jobs=50,
                                scoring=f1_scorer)
        #Sn_sc = cross_val_score(PL0,Xi,Yi, 
        #                        cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
        #                        n_jobs=50,
        #                        scoring=sn_scorer)
        #Sp_sc = cross_val_score(PL0,Xi,Yi, 
        #                        cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
        #                        n_jobs=50,
        #                        scoring=sp_scorer)
        #PPV_sc= cross_val_score(PL0,Xi,Yi, 
        #                        cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
        #                        n_jobs=50,
        #                        scoring=ppv_scorer)
        P.update(dict(f1_mn=np.mean(F1_sc),f1_sd=np.std(F1_sc),
                      #sn_mn=np.mean(Sn_sc),sn_sd=np.std(Sn_sc),
         #             ppv_mn=np.mean(PPV_sc),ppv_sd=np.std(PPV_sc),
                      #sp_mn=np.mean(Sp_sc),sp_sd=np.std(Sp_sc)
                     ))
        print "\t    F1 %(f1_mn)3.2f(%(f1_sd)3.2f)" % P
        #print "\t    PPV %(ppv_mn)3.2f(%(ppv_sd)3.2f)\n" % P
        PERF_r.append(P)
    

	 SUB bio (398, 373) 496 31
	    F1 0.70(0.30)
	 SUB bc (392, 476) 496 31
	    F1 0.93(0.03)
	 SUB bct (392, 433) 496 31
	    F1 0.80(0.27)
	 SUB chm (437, 114) 496 31
	    F1 0.26(0.16)
	 SUB ct (392, 68) 496 31
	    F1 0.46(0.32)
	 CHR bio (428, 369) 520 19
	    F1 0.69(0.36)
	 CHR bc (421, 471) 520 19
	    F1 0.94(0.03)
	 CHR bct (421, 435) 520 19
	    F1 0.54(0.41)
	 CHR chm (474, 109) 520 19
	    F1 0.22(0.33)
	 CHR ct (421, 67) 520 19
	    F1 0.17(0.35)
	 DEV bio (429, 376) 526 55
	    F1 0.28(0.18)
	 DEV bc (424, 481) 526 55
	    F1 0.38(0.26)
	 DEV bct (424, 439) 526 55
	    F1 0.32(0.03)
	 DEV chm (476, 115) 526 55
	    F1 0.11(0.07)
	 DEV ct (424, 64) 526 55
	    F1 0.25(0.34)
	 DNT bio (79, 423) 92 19
	    F1 0.87(0.05)
	 DNT bc (76, 588) 92 19
	    F1 0.85(0.09)
	 DNT bct (76, 478) 92 19
	    F1 0.77(0.09)
	 DNT chm (100, 114) 92 19
	    F1 0.80(0.06)
	 DNT ct (76, 61) 92 19
	    F1 0.86(0.06)
	 MGR bio (346, 383) 338 101
	    F1 0.34(0.21)
	 MGR bc (339, 508) 338 101
	    

In [63]:
tox = 'DNT'
dt  = 'bct'
DS1 = getToxDataSet(tox,MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')
N_p,N_n = (DS1['tox'][tox]==1).sum(),(DS1['tox'][tox]==0).sum()
        
I = DS1['tox'].index.intersection(DS1[dt].index)
Xi,Yi = DS1[dt].ix[I], DS1['tox'].ix[I]
Fi = Xi.sum(axis=0)/Xi.shape[0]
Xi = Xi[Fi[Fi>0.05].index]
        
neg_f = 1.0*N_n/(N_n+N_p)
P = dict(cls=tox,dti=dt,cv='kf5')
print "\t %(cls)s %(dti)s" % P, Xi.shape,N_p,N_n

for i in range(10):
    Yr = Yi.copy()
    Ys = Yi.sample(frac=0.3)
    Yr.ix[Ys.index]=0
    Yr.ix[Yr.index.difference(Ys.index)]=1

    PL0 = Pipeline([('fsel',SelectKBest(f_classif, k=12)),
                    ('svc',SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0, 
                       degree=3,gamma=10.0, kernel='rbf', max_iter=100, 
                       probability=False, shrinking=True,
                       tol=0.001, verbose=False))])

    F1_sc = cross_val_score(PL0,Xi,Yr, 
                            cv=StratifiedKFold(np.array(Yi[tox]),n_folds=5),
                            n_jobs=50,
                            scoring=f1_scorer)
    
    P.update(dict(f1_mn=np.mean(F1_sc),f1_sd=np.std(F1_sc)))
    print "\t    F1 %(f1_mn)3.2f(%(f1_sd)3.2f)" % P

	 DNT bct (76, 478) 92 19
	    F1 0.46(0.31)
	    F1 0.60(0.16)
	    F1 0.67(0.12)
	    F1 0.77(0.06)
	    F1 0.66(0.21)
	    F1 0.72(0.07)
	    F1 0.62(0.10)
	    F1 0.64(0.12)
	    F1 0.65(0.11)
	    F1 0.52(0.14)


In [61]:
neg_f

0.17117117117117117